# Downloading and preprocessing region growth anotations

Region Growth anotations can be downloanded from https://github.com/observethesun/coronal_holes/blob/mnras2018/data/CH_abp.zip



### Data reference

Illarionov, E., & Tlatov, A. (2018). 
Segmentation of coronal holes in solar disc images with a convolutional neural network.
Monthly Notices of the Royal Astronomical Society, 481(4), 5014–5021.

Tlatov, A., Tavastsherna, K., & Vasil’eva, V. (2014). 
Coronal Holes in Solar Cycles 21 to 23.
Solar Physics, 289.

In [40]:
import sys, os
sys.path.append('../src')

In [41]:
import glob
import zipfile
from datetime import datetime
from pathlib import Path

import astropy.units as u
import numpy as np
import sunpy
from PIL import Image, ImageDraw
from sunpy.coordinates.sun import angular_radius
from sunpy.net.helioviewer import HelioviewerClient
from tqdm import tqdm

from prep_utils import (convert_jp2_to_png, create_labels_img,
                        create_limb_mask, crop_limb)

In [42]:
# Unzip sample data
with zipfile.ZipFile("../data/sample_region_growth_anotation.zip","r") as zip_ref:
    zip_ref.extractall("../data/")

In [43]:
# Define function for parsing files
def get_objects(path, types=None):
    """Read pixel coordinates of objects from abp file."""
    spots = []
    cores = []
    with open(path, 'r') as fin:
        fread = fin.readlines()
        size = fread[0].split(" ")[0:2]
        num_objects = int(fread[2])
        for i in range(num_objects):
            info = np.array(fread[3 + 2 * i].split()).astype('int')
            if (info[-1] == 0) and (types is None):
                data = fread[4 + 2 * i].split()
                spots.extend(data)
            elif (info[-1] == 0) and ('spots' in types):
                data = fread[4 + 2 * i].split()
                spots.extend(data)
            elif (info[-1] != 0) and ('cores' in types):
                data = fread[4 + 2 * i].split()
                cores.extend(data)
            else:
                continue
    object_list = []
    if types is None:
        spots = np.array(spots).reshape((-1, 3))[:, 1:None:-1].astype('int')
        object_list.append(spots)
    else:
        if 'spots' in types:
            spots = np.array(spots).reshape((-1, 3))[:, 1:None:-1].astype('int')
            object_list.append(spots)
        if 'cores' in types:
            cores = np.array(cores).reshape((-1, 3))[:, 1:None:-1].astype('int')
            object_list.append(cores)
    return size, object_list

In [44]:
coord_files = glob.glob("../data/region_growth_anotation/*.abp")
dir_path = "../data/193/"
# Create required folders
Path(dir_path + "/masks/").mkdir(parents=True, exist_ok=True)
hv = HelioviewerClient()

for file in tqdm(coord_files):
    date = file.split("\\")[-1].replace("_aia.abp", "")
    date = datetime.strptime(date, '%Y%m%d_%H%M%S')
    # Download image file from HEK
    fname = hv.download_png(date.strftime("%Y/%m/%d %H:%M:%S"), 2.4, "[SDO,AIA,AIA,193,1,100]",
                            x0=0, y0=0, width=1024, height=1024, progress=False,
                            directory=f"{dir_path}/imgs/")
    fname = fname.split("\\")[-1]
    # Get coords from file
    size, obj = get_objects(file)
    coords = []
    for point in obj[0]:
        x = int(point[0])
        y = int(point[1])
        coords.append((x, y))
    # Create empty segmentation mask
    im = Image.new('RGB', (1024, 1024), (0, 0, 0))
    img = ImageDraw.Draw(im)
    # Draw segmentations
    for annotation in coords:
        img.point(annotation, fill="white")
    im = im.transpose(method=Image.FLIP_TOP_BOTTOM)
    im = im.rotate(270)
    im.save(dir_path + "/masks/" + fname)

100%|██████████| 15/15 [02:00<00:00,  8.01s/it]


In [51]:
# Create limb mask
create_limb_mask(img_size=(1024, 1024), radius=404, name=dir_path + "limb_mask.png")

# Crop limb and save as new img
imgs = glob.glob(f"{dir_path}/imgs/*.png")
limb = Image.open(dir_path + "limb_mask.png").convert("RGB")
Path(dir_path + "/imgs_prep/").mkdir(parents=True, exist_ok=True)

for img in tqdm(imgs):
    fname = img.split("\\")[-1]
    img = Image.open(img).convert("RGB")
    cropped = crop_limb(img, limb)
    cropped.save(f"{dir_path}/imgs_prep/{fname}")

100%|██████████| 15/15 [00:09<00:00,  1.57it/s]
